In [1]:
import pandas as pd

In [3]:
loans = pd.read_csv('airflow_home/loanstats.csv')

/Users/emily-air13/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
loans.head()

In [4]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

missing_percents = (len(loans.index) - loans.count())/len(loans.index)
#missing_percents.sort_values(inplace=True)
#missing_percents.plot.barh(figsize=(20,40))
#plt.yticks(size=20)
#plt.show()



In [5]:
drops = missing_percents[missing_percents > .7]

In [6]:
drops

id                                            0.999986
member_id                                     1.000000
url                                           1.000000
desc                                          0.923447
mths_since_last_record                        0.831040
mths_since_last_major_derog                   0.734741
annual_inc_joint                              0.979042
dti_joint                                     0.979044
verification_status_joint                     0.979042
mths_since_recent_bc_dlq                      0.758860
revol_bal_joint                               0.986747
sec_app_earliest_cr_line                      0.986747
sec_app_inq_last_6mths                        0.986747
sec_app_mort_acc                              0.986747
sec_app_open_acc                              0.986747
sec_app_revol_util                            0.986922
sec_app_open_act_il                           0.986747
sec_app_num_rev_accts                         0.986747
sec_app_ch

In [7]:
columns = drops.index

In [8]:
for c in columns:
    loans.drop(c, axis=1, inplace=True)

In [ ]:
loans.dtypes

In [ ]:
loans.count()

In [9]:
pd.set_option('display.max_columns', 110)

In [ ]:
loans.describe()

In [10]:
t = loans.isnull().any()
t[t==False]

set          False
timestamp    False
dtype: bool

In [11]:
loans_temp = loans.drop(['set', 'timestamp'], axis=1)

In [12]:
t2 = loans_temp.isnull().all(1)
b = t2[t2==True].index


In [13]:
loans.drop(loans.index[b], inplace=True)

In [ ]:
declined = pd.read_csv('airflow_home/declinestats.csv')

In [ ]:
declined.head()

In [ ]:
missing_percentsd = (len(declined.index) - declined.count())/len(declined.index)
missing_percentsd.sort_values(inplace=True)
missing_percentsd

In [ ]:
declined['Risk_Score'].describe()

In [ ]:
l = list(loans)
d = list(declined)
l
#res = list(set(l)^set(d))
#res

In [ ]:
d



'Amount Requested',
 'Application Date',
 'Loan Title',
 'Risk_Score',
 'Debt-To-Income Ratio',
 'Zip Code',
 'State',
 'Employment Length',
 'Policy Code',
 'set',
 'timestamp'


In [ ]:
t = loans['funded_amnt'] - loans['loan_amnt']

In [ ]:
t.describe()

In [ ]:
t = loans[loans['funded_amnt'] != loans['loan_amnt']]

In [ ]:
t.count()

In [ ]:
t['loan_amnt'].describe()

In [ ]:
t['funded_amnt'].describe()

In [ ]:
types = loans.dtypes
types = types[types!=object]
types

In [ ]:
numerical = loans[types.index]


import seaborn as sns


plt.close('all')
plt.figure(figsize=(20,20))
sns.heatmap(numerical.corr(), vmax=.8, square=True, cmap=cm.coolwarm)
plt.show()

In [ ]:
loans['grade'].value_counts()

In [ ]:
declined['Risk_Score'].max()

In [ ]:
t = declined[declined['Risk_Score'] > 850]
t

In [ ]:
declined.count()

In [ ]:
declined['Risk_Score'].mean()

In [ ]:
import datetime
format = '%Y-%m-%d'
declined['Application Date'] = declined['Application Date'].map(lambda a: datetime.datetime.strptime(a, format))


In [ ]:
m1=(declined['Application Date'] >='2013-11-06')
#m2=(declined['Application Date'] <'2013-11-06')

declined.loc[m1,'Risk_Score']=declined.loc[m1,'Risk_Score'] * (85/99)
#declined.loc[m2,'Risk_Score']=declined.loc[m2,'Risk_Score']

In [ ]:
declined['Risk_Score'].count()

In [ ]:
declined['Risk_Score'].fillna(declined['Risk_Score'].mean(), inplace=True)

Option 1: Drop Rows without a Credit Score
Option 2: Fill NaNs

In [ ]:
#declined['Debt-To-Income Ratio'].fillna(declined['Debt-To-Income Ratio'].mean(), inplace=True)
declined['Debt-To-Income Ratio'].mean()

### FICO for Loan Stats

In [15]:
len = loans['set'].count()
len

1646778

In [21]:
SIGMA=100.0 # stddev for FICO obtained from here: http://www.fico.com/en/blogs/risk-compliance/us-credit-quality-continues-climb-will-level/ in the comments section
import scipy.stats as stats

a, b = 600, 850
mu, sigma = 699, 100
dist = stats.truncnorm((a - mu) / sigma, (b - mu) / sigma, loc=mu, scale=sigma)

values = dist.rvs(1646778)
values.mean()


714.0741361646991

In [25]:
SIGMA=100.0 # stddev for FICO obtained from here: http://www.fico.com/en/blogs/risk-compliance/us-credit-quality-continues-climb-will-level/ in the comments section
import random

def generate_number(average):
    x = random.normalvariate(average, SIGMA)
    while x > 599 or x < 851:
        x = random.normalvariate(average, SIGMA)
    return x



In [ ]:
v = []
for i in range(1646778):
    v.append(generate_number(699))

In [ ]:
v2 = []
for i in range(1646778):
    v2.append(random.triangular(600, 850, 699))